In [52]:
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, \
PowerTransformer, KBinsDiscretizer,FunctionTransformer
from sklearn.decomposition import PCA
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectKBest, chi2

import seaborn as sns
import matplotlib.pyplot as plt
import joblib

url= "https://proai-datasets.s3.eu-west-3.amazonaws.com/sample_dataset.csv"

df = pd.read_csv(url)

features = df.drop('target', axis=1)
numeric= df.select_dtypes(exclude='object')
categoriche = features.select_dtypes(include='object').columns.tolist()


In [ ]:
"""  Pipeline 1  """


#   -->    Funzione per filtrare le colonne numeriche con skewness oltre il limite stabilito di +-1 (parametro variabile)
def Skewness_filter(dataset, threshold=1):

    skewness = dataset.skew(numeric_only=True)
    asymmetric = skewness[(skewness > threshold) | (skewness < -threshold)].index.tolist()
    symmetric = skewness[(skewness <= threshold) & (skewness >= -threshold)].index.tolist()
    return asymmetric, symmetric

#  -->     Funzione per applicare pipeline differenti in base al tipo di colonna
def skew_pipeline(column_type):

    if column_type == 'asymmetric':
        return Pipeline([
            ('cleaning', SimpleImputer(strategy='mean')),      
            ('Yeo-Johnson', PowerTransformer()),
            ('standard', StandardScaler())
        ])
    elif column_type == 'symmetric':
        return Pipeline([
            ('cleaning', SimpleImputer(strategy='mean')),
            ('standard', StandardScaler())
        ])

#   -->    funzione 1 per dividere e restituire le colonne in base alla simmetria o meno (Skewness_filter)
#   -->    funzione 2 per applicarla nel FunctionTransformer
def column_type_assigner(X, threshold=1):
    asimmetriche, simmetriche = Skewness_filter(pd.DataFrame(X), threshold=threshold)
    return pd.DataFrame(X), asimmetriche, simmetriche

def assign_columns(X, threshold=1):
    return column_type_assigner(X, threshold)[0]



#   -->   Funzione 1 per comporre il dataframe finale ordinato (utimo passaggio Pipeline)
#   -->   Funzione 2 per applicarla nel FunctionTransformer finale

def final_transformer(X, df, asimmetriche, simmetriche):
 
    transformed_data = pd.DataFrame(X, columns=['A', 'B', 'C'] + asimmetriche + simmetriche)   
    transformed_data['target'] = df['target'].values   
    ordered_df = df[asimmetriche + simmetriche + ['target']]
    #riprende i valori originali per target != 1 e colomme numeriche 
    indices_to_replace = ordered_df.index[ordered_df['target'] != 1]
    transformed_data.loc[indices_to_replace, asimmetriche + simmetriche] = \
    ordered_df.loc[indices_to_replace, asimmetriche + simmetriche] 
    return transformed_data

def apply_final_transform(X):
    return final_transformer(X, df, asimmetriche, simmetriche)



#INIZIO PIPELINE 
pipeline = ColumnTransformer([
        ('categoriche', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'))
         ]), categoriche),
    ('asimmetriche', skew_pipeline('asymmetric'), asimmetriche),
    ('simmetriche', skew_pipeline('symmetric'), simmetriche)
])

complete_pipeline = Pipeline([
    ('assign_columns', FunctionTransformer(func=assign_columns, validate=False)),
    ('column_transform', pipeline),
    ('final_transform', FunctionTransformer(
    func=apply_final_transform, validate=False))
])


transformed_data = complete_pipeline.fit_transform(features)
transformed_data



In [50]:
"""  Pipeline 2  
Applicare, a tutti i record del dataset:
Pulizia dei valori mancanti con procedere a tua scelta
Discretizzazione a 20 bin di tutte le variabili numeriche
Encoding ordinale per la variabile categorica secondo i valori, in ordine crescente: A, B, C
Selezione delle 5 variabili più informative rispetto al target fornito
"""

y = df['target']

pipeline2 = ColumnTransformer([
        ('numeriche', Pipeline([ ('cleaning',KNNImputer(n_neighbors=5, weights='distance')),
                                 ('Dicretizer', KBinsDiscretizer(strategy='quantile', n_bins=20, encode='ordinal'))
                               ]), make_column_selector(dtype_exclude=['object','bool','category'])),
        
        ('categoriche', Pipeline([
                                    ('cleaning', SimpleImputer(strategy='most_frequent')),
                                    ('onehot', OrdinalEncoder( categories=[["A","B","C"]]))
                                    ]), make_column_selector(dtype_include=['object','bool','category']))
])

#funzione per scegliere le migliori 5 feature (chi2 perchè lavoriamo con categoriche su target categorico)
selection= SelectKBest(chi2, k=5)

final_pipeline= Pipeline([ ('pre_processing', pipeline2),
                              ('selection', selection )
])

final_pipeline.fit_transform(features,y)

#per vedere le 5 feature scelte dal test chi2
selected= features.columns[final_pipeline['selection'].get_support()]
selected


Index(['mean concave points', 'fractal dimension error', 'worst texture',
       'worst perimeter', 'worst concavity'],
      dtype='object')

In [ ]:

"""Pipeline 3 
Applicare, a tutti i record del dataset e alle sole variabili numeriche, la seguente pipeline:
Pulizia dei valori mancanti mediante metodo a tua scelta
Principal Component Analysis all’80% di varianza spiegata
Simmetrizzazione
Riscalatura mediante normalizzazione tra 0 e 1
Esportare la pipeline finale."""


pipeline3 = ColumnTransformer([
        ('numeriche', Pipeline([ ('cleaning',SimpleImputer(strategy='median')),
                                 ('pca', PCA(n_components=0.8)),
                                 ('Yeo-Johnson', PowerTransformer()),
                                 ('minmax', MinMaxScaler())
                               ]), 
                    make_column_selector(dtype_exclude=['object','bool','category']))
])

muted = pipeline3.fit_transform(features)
joblib.dump(pipeline3, 'pipeline3.joblib')
